# [SOC-88] Mapping Crime in San Francisco

### Professor David Harding

## Table of Contents

[Introduction](#intro)

[The Data](#data)

[Base Maps](#1)
   - [Question 1](#q1)

[Markers](#2)
   - [Question 2](#q2)
   - [Question 3](#q3)
   - [Question 4](#q4)
    
[Choropleth Maps](#3)
   - [Question 5](#q5)
   - [Question 6](#q6)



## Introduction <a id='intro'></a>

In this homework, you will practice different data mapping techniques you learned about in lecture and lab. The data has been taken from [SF Data](https://data.sfgov.org/), San Francisco's open data site. 

There are two main data files used in this assignment: **SFPD_incidents_2020.csv** and **sfpd-police-districts.geojson**. 

The first file, **SFPD_incidents_2020.csv**, has records of all police incidents that took place in 2016. Its columns contain information such as the latitude-longitude information of incidents, police precinct and neighborhood in which the incident occurred, time and date of the report, type of crime, etc. 

The second file, **sfpd-police-districts.geojson**, contains geographic information about the boundaries of San Francisco police districts. These boundaries are necessary for making choropleth plots.

---


We will begin by running a code cell that will load the libraries you'll be using.

In [20]:
# load the necessary software
from datascience import *
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import datetime
import folium
import json

## The Data <a id='data'></a>


Our main dataset comes from the [city of San Francisco's open data portal](https://data.sfgov.org/Public-Safety/Police-Department-Incident-Reports-2018-to-Present/wg3w-h783). 

Run the next cell to load the incident data.

In [21]:
# load SF Police Incident Data, 2020
incidents = Table().read_table('Data/SFPD_incidents_2020.csv')
incidents.show(5)

Incident Datetime,Incident Date,Incident Time,Incident Year,Incident Day of Week,Report Datetime,Row ID,Incident ID,Incident Number,CAD Number,Report Type Code,Report Type Description,Filed Online,Incident Code,Incident Category,Incident Subcategory,Incident Description,Resolution,Intersection,CNN,Police District,Analysis Neighborhood,Supervisor District,Latitude,Longitude,point
2020/01/07 04:00:00 AM,2020/01/07,04:00,2020,Tuesday,2020/02/06 09:41:00 AM,90012406372,900124,206027460,nan,II,Coplogic Initial,1,6372,Larceny Theft,Larceny Theft - Other,"Theft, Other Property, $50-$200",Open or Active,nan,nan,Out of SF,nan,nan,nan,nan,nan
2020/01/31 03:00:00 PM,2020/01/31,15:00,2020,Friday,2020/01/31 03:00:00 PM,89876863010,898768,190783352,nan,IS,Initial Supplement,nan,63010,Warrant,Other,"Warrant Arrest, Local SF Warrant",Cite or Arrest Adult,nan,nan,Out of SF,nan,nan,nan,nan,nan
2020/04/07 10:00:00 AM,2020/04/07,10:00,2020,Tuesday,2020/04/07 09:39:00 PM,95329775025,953297,200227624,2.00983e+08,IS,Initial Supplement,nan,75025,Non-Criminal,Non-Criminal,Search Warrant Service,Cite or Arrest Adult,nan,nan,Out of SF,nan,nan,nan,nan,nan
2020/02/12 02:00:00 PM,2020/02/12,14:00,2020,Wednesday,2020/02/13 02:30:00 PM,90267006244,902670,206033291,nan,II,Coplogic Initial,1,6244,Larceny Theft,Larceny - From Vehicle,"Theft, From Locked Vehicle, >$950",Open or Active,nan,nan,Northern,nan,nan,nan,nan,nan
2020/01/13 08:10:00 PM,2020/01/13,20:10,2020,Monday,2020/01/13 09:29:00 PM,90358806224,903588,206035407,nan,II,Coplogic Initial,1,6224,Larceny Theft,Larceny - From Vehicle,"Theft, From Unlocked Vehicle, >$950",Open or Active,nan,nan,Richmond,nan,nan,nan,nan,nan


Each row in this table represents a different incident reported to the San Francisco Police Department (SFPD). Most of the columns are fairly intuitive, but we'll narrow down to a few that are of particular interest:

- `Incident Number`and `Incident ID` are ID numbers to identify each different incident, used for organization within the police department.

- `Incident Category` classifies the incident as of 49 types. We can see all possible categories using the `group` method.

In [22]:
# show the unique categories from 'Incident Category'
incidents.group("Incident Category").show()

Incident Category,count
Arson,197
Assault,3347
Burglary,4078
Case Closure,176
Civil Sidewalks,25
Courtesy Report,137
Disorderly Conduct,982
Drug Offense,1356
Drug Violation,19
Embezzlement,85


- `Incident Description` gives more information on what occurred during the incident. You can think of the `Incident Description` as a subtype of the categories in `Incident Category`. Not only are there too many unique descriptions to list them all, but it's too descriptive for what we want to analyze later in this homework. It makes more sense to select a particular category and then list the possible descriptions for only that category. In the next cell, you can view the possible `Incident Description` values for incidents falling under the category `"Non-Criminal"`.

In [23]:
# show the unique incident descriptions for the 'Non-Criminal' category
incidents.where("Incident Category", "Non-Criminal").group('Incident Description').show()

Incident Description,count
Aided Case,528
Aided Case -Property for Destruction,27
"Aided Case, Injured or Sick Person",66
"Aided Case, Sick Person",9
"Aided case, Naloxone Deployment",132
"Dog, Barking",1
"Dog, Bite or Attack",104
"Dog, Stray or Vicious",4
"Firearm, Turned In by Public",37
Found Property,763


- `Resolution` gives information about what the police did for the respective incident. Once again, we can view all possible resolution options using the `group` method

In [24]:
# show the unique resolutions
incidents.group("Resolution").show()

Resolution,count
Cite or Arrest Adult,11945
Exceptional Adult,185
Open or Active,45652
Unfounded,260


- Finally, `Latitude`, `Longitude`, and `point` give geographic data about the incident. Think of `Latitude` as your `Y` and `Longitude` as your `X`. `point` has both the latitude and longitude together as a pair of integers.

## Base Maps <a id='1'></a>

### Question 1 <a id='q1'></a>
Create a base map centered on San Francisco. Choose an appropriate zoom start and tiles.

*Note: we're going to be creating several maps in this homework, so it's easier to create variables for the starting coordinates, zoom, and tiles, and use them over and over again, rather than rewrite them in every map we make.*

In [25]:
# add coordinates for San Francisco
sf_coordinates = [37.7749,-122.4194]
sf_zoom_start = 12
sf_tiles = 'cartodbpositron'

# create a map of San Francisco
sf_map = folium.Map(location=sf_coordinates, zoom_start=sf_zoom_start, tiles=sf_tiles)
sf_map

## Markers <a id='2'></a>

In the next two cells, we've isolated two police incidents for you.

In [26]:
incidentA = incidents.where("Incident ID", 922733)
incidentA

Incident Datetime,Incident Date,Incident Time,Incident Year,Incident Day of Week,Report Datetime,Row ID,Incident ID,Incident Number,CAD Number,Report Type Code,Report Type Description,Filed Online,Incident Code,Incident Category,Incident Subcategory,Incident Description,Resolution,Intersection,CNN,Police District,Analysis Neighborhood,Supervisor District,Latitude,Longitude,point
2020/04/26 09:00:00 PM,2020/04/26,21:00,2020,Sunday,2020/04/27 03:20:00 PM,92273306244,922733,200263101,2.01182e+08,II,Initial,nan,6244,Larceny Theft,Larceny - From Vehicle,"Theft, From Locked Vehicle, >$950",Open or Active,HARKNESS AVE \ SAN BRUNO AVE,2.0556e+07,Ingleside,Visitacion Valley,10,37.7179,-122.4,"(37.717906309071864, -122.40013147694049)"


In [27]:
incidentB = incidents.where("Incident ID", 886770)
incidentB

Incident Datetime,Incident Date,Incident Time,Incident Year,Incident Day of Week,Report Datetime,Row ID,Incident ID,Incident Number,CAD Number,Report Type Code,Report Type Description,Filed Online,Incident Code,Incident Category,Incident Subcategory,Incident Description,Resolution,Intersection,CNN,Police District,Analysis Neighborhood,Supervisor District,Latitude,Longitude,point
2020/01/01 02:41:00 AM,2020/01/01,02:41,2020,Wednesday,2020/01/01 02:41:00 AM,88677026031,886770,200000389,2.00011e+08,II,Initial,nan,26031,Arson,Arson,Arson of Vehicle,Open or Active,23RD ST \ CAROLINA ST,3.3046e+07,Bayview,Potrero Hill,10,37.7547,-122.4,"(37.75469183949377, -122.39957958699718)"


### Question 2 <a id='q2'></a>
Create a marker for each of the above incidents, including:
* incident location
* an appropriate and informative pop-up (appears when you hover over the marker)
* an appropriate and informative tooltip (appears when you click on the marker)
* an appropriate color and type for the icon, given the type of incident

You can view the list of icon options at https://getbootstrap.com/docs/3.3/components/

In [28]:
# a clean map for the markers
marker_map = folium.Map(location=sf_coordinates, zoom_start=sf_zoom_start, tiles=sf_tiles)

# For Incident A
coordinateA = [37.717906309071864, -122.40013147694049]
popupA = 'Larceny - From Vehicle'
tooltipA = 'Theft, From Locked Vehicle, >$950'
folium.Marker(location=coordinateA, popup=popupA, 
              tooltip=tooltipA, icon=folium.Icon(color = 'green', icon = 'fa-solid fa-car',prefix = 'fa')).add_to(marker_map)

# view the map
marker_map

In [29]:
# For Incident B
coordinateB = [37.75469183949377, -122.39957958699718]
popupB = "Arson"
tooltipB = "Arson of Vehicle"
folium.Marker(location=coordinateB, popup=popupB, 
              tooltip=tooltipB, icon=folium.Icon(color= 'red', icon='fa-solid fa-fire', prefix = 'fa')).add_to(marker_map)
marker_map

Next, we'd like to map all incidences of disorderly conduct.

First, we make a table that only contains disorderly conduct incidents.

In [30]:
# filter for just disorderly conduct
disorderly = incidents.where("Incident Category", "Disorderly Conduct")
disorderly.show(3)

Incident Datetime,Incident Date,Incident Time,Incident Year,Incident Day of Week,Report Datetime,Row ID,Incident ID,Incident Number,CAD Number,Report Type Code,Report Type Description,Filed Online,Incident Code,Incident Category,Incident Subcategory,Incident Description,Resolution,Intersection,CNN,Police District,Analysis Neighborhood,Supervisor District,Latitude,Longitude,point
2020/02/19 08:57:00 PM,2020/02/19,20:57,2020,Wednesday,2020/02/19 09:15:00 PM,96214819057,962148,200127290,2.00504e+08,IS,Initial Supplement,nan,19057,Disorderly Conduct,Intimidation,Terrorist Threats,Exceptional Adult,ANDOVER ST \ RICHLAND AVE,2.12e+07,Ingleside,Bernal Heights,9,37.7356,-122.417,"(37.73560466456792, -122.41675952114524)"
2020/01/01 12:20:00 AM,2020/01/01,00:20,2020,Wednesday,2020/01/01 12:36:00 AM,88675519090,886755,200000094,2.0001e+08,II,Initial,nan,19090,Disorderly Conduct,Drunkenness,"Alcohol, Under Influence Of In Public Place",Open or Active,EDDY ST \ JONES ST,2.4929e+07,Tenderloin,Tenderloin,6,37.7839,-122.413,"(37.7839325760642, -122.41259527758581)"
2020/01/01 12:20:00 AM,2020/01/01,00:20,2020,Wednesday,2020/01/01 12:36:00 AM,88679519090,886795,200000094,2.0001e+08,IS,Initial Supplement,nan,19090,Disorderly Conduct,Drunkenness,"Alcohol, Under Influence Of In Public Place",Cite or Arrest Adult,EDDY ST \ JONES ST,2.4929e+07,Tenderloin,Tenderloin,6,37.7839,-122.413,"(37.7839325760642, -122.41259527758581)"


In [31]:
# This cell removes all the disorderly conduct incidents that didn't report a location
import math
no_loc_incidents = []
for i in range(disorderly.num_rows):
    incident = disorderly.take(i)
    if math.isnan(incident.column('Latitude').item(0)):
        no_loc_incidents.append(i)
disorderly.remove(no_loc_incidents)
disorderly.show(3)

Incident Datetime,Incident Date,Incident Time,Incident Year,Incident Day of Week,Report Datetime,Row ID,Incident ID,Incident Number,CAD Number,Report Type Code,Report Type Description,Filed Online,Incident Code,Incident Category,Incident Subcategory,Incident Description,Resolution,Intersection,CNN,Police District,Analysis Neighborhood,Supervisor District,Latitude,Longitude,point
2020/02/19 08:57:00 PM,2020/02/19,20:57,2020,Wednesday,2020/02/19 09:15:00 PM,96214819057,962148,200127290,2.00504e+08,IS,Initial Supplement,nan,19057,Disorderly Conduct,Intimidation,Terrorist Threats,Exceptional Adult,ANDOVER ST \ RICHLAND AVE,2.12e+07,Ingleside,Bernal Heights,9,37.7356,-122.417,"(37.73560466456792, -122.41675952114524)"
2020/01/01 12:20:00 AM,2020/01/01,00:20,2020,Wednesday,2020/01/01 12:36:00 AM,88675519090,886755,200000094,2.0001e+08,II,Initial,nan,19090,Disorderly Conduct,Drunkenness,"Alcohol, Under Influence Of In Public Place",Open or Active,EDDY ST \ JONES ST,2.4929e+07,Tenderloin,Tenderloin,6,37.7839,-122.413,"(37.7839325760642, -122.41259527758581)"
2020/01/01 12:20:00 AM,2020/01/01,00:20,2020,Wednesday,2020/01/01 12:36:00 AM,88679519090,886795,200000094,2.0001e+08,IS,Initial Supplement,nan,19090,Disorderly Conduct,Drunkenness,"Alcohol, Under Influence Of In Public Place",Cite or Arrest Adult,EDDY ST \ JONES ST,2.4929e+07,Tenderloin,Tenderloin,6,37.7839,-122.413,"(37.7839325760642, -122.41259527758581)"


### Question 3 <a id='q3'></a>
Fill in the code below to create markers for all disorderly conduct incidents.

As in question 2, choose the appropriate coordinates, popup, tooltip, color, and icon for the type of incident.

In [32]:
disorderly['Latitude']

[37.73560466456792,
 37.7839325760642,
 37.7839325760642,
 37.788942623294176,
 37.788986292481376,
 37.75117875536422,
 37.78331088878137,
 37.714286975792135,
 37.786924665750064,
 37.73335135825967,
 37.71065338129637,
 37.775558160382,
 37.78016202513452,
 37.78258503232177,
 37.76657054026182,
 37.76357160254071,
 37.73618212314209,
 37.78289412114938,
 37.78742932998812,
 37.77728273405221,
 37.775558160382,
 37.79057825823802,
 37.79594540193314,
 37.78748711637317,
 37.718834357405456,
 37.78549155926893,
 37.74388333564679,
 37.78328785199654,
 37.79885752312248,
 37.752720740441546,
 37.74312376235638,
 37.78351564008561,
 37.78735926098589,
 37.7245877050746,
 37.72990988256753,
 37.80403345433174,
 37.7528356110411,
 37.7681396823622,
 37.78117721773047,
 37.7310799713596,
 37.752795583046776,
 37.722657955936725,
 37.78117721773047,
 37.77346692060748,
 37.716134500935816,
 37.76560583760297,
 37.74004455201744,
 37.7814979983372,
 37.76607460663705,
 37.77210825726729,
 3

In [33]:
# create a clean map for the disorderly conduct incidents
disorderly_map = folium.Map(location=sf_coordinates, tiles=sf_tiles, zoom_start=sf_zoom_start)

# make a marker for each disorderly conduct incident
for i in range(disorderly.num_rows):
    incidentC = disorderly.take(i)
    coordinateC = [incidentC['Latitude'], incidentC['Longitude']]
    popupC = incidentC['Incident Category']
    tooltipC = incidentC['Incident Description']
    folium.Marker(location= coordinateC, popup=popupC, 
                  tooltip=tooltipC, icon=folium.Icon(color='orange', icon='glyphicon glyphicon-user')).add_to(disorderly_map)
    
# show the map
disorderly_map

### Question 4 <a id='q4'></a>

Describe the features you chose for questions 1, 2, and 3, including:
* map tiles
* marker icon
* marker color
* marker popup and tooltip

Why were those features good for the data in those questions?

For the map tiles, I wanted to make sure that the map itself was not overpowering. As the map itself is simply just a way to identify the location of the incidents.
For the marker icons, I wanted to choose markers that kind of relate to the type of incident that occured.
For example, car theft incidents were labeled with cars, arson incidents with fire, and disorderly conduct incidents with a picture of a person.
For marker colors, I wanted to make sure that each marker colors were different to differentiate different types of incidents. For example, arson incidents with fire 

## Choropleth maps <a id='3'></a>

In this section, you're going to create a choropleth map with the number of non-criminal mental health-related incidents in each police district.

First, we need to load the geojson file that gives the boundaries for the police districts. Run the next cell to load the geojson.

In [34]:
# load SFPD district boundaries
sf_districts = json.load(open('Data/sf-police-districts.geojson'))
sf_districts

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'shape_area': '91344142.1925',
    'shape_leng': '87550.2751419',
    'company': 'B',
    'shape_le_1': '100231.353916',
    'district': 'SOUTHERN'},
   'geometry': {'type': 'MultiPolygon',
    'coordinates': [[[[-122.39186138812263, 37.79424680970543],
       [-122.39178965045961, 37.7941712252344],
       [-122.39173396937666, 37.794202997266886],
       [-122.39166681055336, 37.794132183335286],
       [-122.39172264479394, 37.79410052824544],
       [-122.3916726266299, 37.79404791882388],
       [-122.39198184041989, 37.79387175411522],
       [-122.39158906053727, 37.7935276602537],
       [-122.391723729351, 37.79342884992162],
       [-122.39081210865645, 37.79266254752923],
       [-122.38929874837604, 37.791390503319754],
       [-122.38880784444565, 37.790977844242896],
       [-122.38893019202212, 37.790880005297964],
       [-122.38861191205169, 37.790313282442106],
       [-122.3880494246161

You can see the districts overlaid onto the San Francisco Map by running the next cell.

In [35]:
# make the folium geojson object and add to a map of SF
m = folium.Map(sf_coordinates, zoom_start=sf_zoom_start, tiles=sf_tiles)

folium.GeoJson(
    sf_districts,
    style_function=lambda feature: {
        'fillColor': 'white',
        'color': 'blue',
        'weight': 2,
        'dashArray': '5, 5'
    }
).add_to(m)
m

To make our choropleth overlay, we must first get the counts of mental health incidents by district. First, we use the `where` method to select only the incidents that have a description of "Metal Health Detention".

In [36]:
# mental health related incidents
mental_health = incidents.where("Incident Description", are.equal_to("Mental Health Detention"))
mental_health.show(3)

Incident Datetime,Incident Date,Incident Time,Incident Year,Incident Day of Week,Report Datetime,Row ID,Incident ID,Incident Number,CAD Number,Report Type Code,Report Type Description,Filed Online,Incident Code,Incident Category,Incident Subcategory,Incident Description,Resolution,Intersection,CNN,Police District,Analysis Neighborhood,Supervisor District,Latitude,Longitude,point
2020/01/01 12:33:00 AM,2020/01/01,00:33,2020,Wednesday,2020/01/01 12:33:00 AM,88676164020,886761,200000135,2.0001e+08,II,Initial,nan,64020,Non-Criminal,Other,Mental Health Detention,Open or Active,ILLINOIS ST \ 16TH ST,2.3575e+07,Southern,Potrero Hill,6,37.767,-122.388,"(37.766957909504505, -122.38816413828668)"
2020/01/01 02:24:00 AM,2020/01/01,02:24,2020,Wednesday,2020/01/01 03:05:00 AM,88678864020,886788,200000107,2.00011e+08,IS,Initial Supplement,nan,64020,Non-Criminal,Other,Mental Health Detention,Open or Active,TAYLOR ST \ MACONDRAY LN,2.5434e+07,Central,Russian Hill,3,37.7994,-122.414,"(37.799366533533515, -122.41402750691888)"
2020/01/01 03:58:00 AM,2020/01/01,03:58,2020,Wednesday,2020/01/01 05:19:00 AM,88679164020,886791,200000583,2.00011e+08,II,Initial,nan,64020,Non-Criminal,Other,Mental Health Detention,Open or Active,MASON ST \ WATER ST,2.5472e+07,Central,North Beach,3,37.8044,-122.413,"(37.80440729201466, -122.41335886309902)"


Next, we use `group` to get the counts of mental health incidents per police district.

In [37]:
# get the counts of mental health incidents by district
mental_health_by_district = mental_health.group("Police District")
mental_health_by_district

Police District,count
Bayview,67
Central,200
Ingleside,110
Mission,192
Northern,138
Out of SF,3
Park,72
Richmond,80
Southern,278
Taraval,127


And finally, we convert the counts to a DataFrame so that it works with the Folium software.

In [38]:
# convert to DataFrame
mental_health_df = mental_health_by_district.to_df()
mental_health_df['Police District'] = mental_health_df['Police District'].str.upper()
mental_health_df.head()

,Police District,count
0,BAYVIEW,67
1,CENTRAL,200
2,INGLESIDE,110
3,MISSION,192
4,NORTHERN,138


### Question 5 <a id='q5'></a>
Complete the following code to create a choropleth overlay showing the counts of mental health incidents for each police district. Choose an appropriate and informative:
* fill color (using a colormap)
* fill opacity
* legend name

You can find colormap options at https://matplotlib.org/gallery/color/colormap_reference.html

In [43]:
# create a clean map for the choropleth
m = folium.Map(sf_coordinates, zoom_start=sf_zoom_start, tiles=sf_tiles)

# create the choropleth overlay
folium.Choropleth(
    geo_data=sf_districts,
    data=mental_health_df,
    columns=['Police District','count'],
    key_on='feature.properties.district',
    fill_color='YlOrRd',
    fill_opacity=0.8,
    bins = 9,
    legend_name= 'Number of Mental Health Incidents'
).add_to(m)

m

### Question 6 <a id='q6'></a>
Explain your design choices for the choropleth map. What options did you consider? What options did you end up choosing, and why? Be sure to reference the context of the data when you explain your choices.

For the choropleth map, some options that I considered was the color of the districts and the amount of bins to show the difference between the districts. The color scheme I chose was from yellow to red, the less amount = lighter color, more amount = darker color. Yellow to Red shows clearly the difference between the districts and is easily distuinguishable between high concentration versus lower concentration. 
I decided to increase the number of bins because I wanted to show the slight differences in the amounts of cases per district. This resulted in more clear-cut and distinguishable areas that is extremely easy to read.
With the legen on the top right corner showing the number from lowest to highest, I think the viewer can easily understand what the choropleth map is trying to show, which is the districts in SF and the amount of mental health cases for each district. 
What is interesting is that different SF districts have different amounts of population. I do not think this map clearly shows the difference in population and simply just shows the number of cases.It is obvious that more people = higher chances of there being more mental health cases, and lower population = less mental health cases.A better design I think for this map would to be, instead of just showing the maount of cases per district, a better way would be to show the amount of mental health cases per person to clearly show the difference between the districts.

----


Data Science Modules: http://data.berkeley.edu/education/modules

Data Science Offerings at Berkeley: https://data.berkeley.edu/academics/undergraduate-programs/data-science-offerings



Notebook developed by: Keeley Takimoto, Harold Cha